In [1]:
from numba import cuda
import pandas as pd
import json
import evaluate
import numpy as np
import requests
from datetime import datetime, timezone, timedelta
import time
from transformers import AutoTokenizer, pipeline
from torch.cuda import device_count
from math import floor

In [9]:
a = [1,2,3,4,5,6,7,8,9,10]
print(np.random.permutation(a))

[ 6  1  7  2  8  4  9  3  5 10]


In [11]:
for i in range(device_count()):
    cuda.select_device(i).reset()

In [2]:
mock_data_path = "../../data/llm-anamnesis/mock_data_2_big.xlsx"
dfs = pd.read_excel(mock_data_path, sheet_name=None, index_col=None)

# Convert json objects to dict
#for table, df in dfs.items():
#    for col in ["schedule_params", "filter", "model_params", "metadata"]:
#        if col in df.columns:
#            df[col] = df[col].apply(lambda x: json.loads(x))

raw_df = dfs["raw"]
config_df = dfs["config"]
summary_df = dfs["summarized"]

In [3]:
print(summary_df.iloc[18]['text'])

Sjukdomshistoria (Patientens diagnoser, sjukdomshistorik och riskfaktorer (t.ex. sjukdomar i familjen)):
    • Man
    • 79 år
    • FF
    • HT
    • DM typ 2
    • NPH
Sökorsaker (patientens symtom och/eller datum för ingrepp):
    • Natt: Diskret förbättring
    • Infektionsvärden: UA
    • CT kontroll: UA
    • P-glukos: Normal

    • Konsult 24-02-05; 17:55
        ◦ Motoriskt orolig
        ◦ Utåtreagerande
        ◦ Ropat x flera
        ◦ Gripit efter saker i luften
        ◦ Vänt på dygnet
        ◦ Sover mycket under dagtid
        ◦ Vaken hela nätterna
        ◦ Ej orienterad i tid och rum
        ◦ Ingen ögonkontakt
        ◦ Motoriskt orolig
        ◦ Förnekar hörselshallucinationer
        ◦ Inga suicidaltankar

Åtgärder (planerade undersökningar, behandlingar och åtgärder)
    • Återinsätter Pradaxa

    • Konsult 24-02-05; 17:55
        ◦ Bedömning: Delirium UNS
        ◦ Tabl Risperidon 0,5 mg x 1
        ◦ Tabl Mirtazapin 15 mg (till natten)
        ◦ Konservativt
   

In [14]:
config_df.head()

,section_id,config_id,active,last_run,schedule_params,filter,model_params,instruction
0,101,101,True,2024-01-01,"{""freq"": ""daily""}","{""unique"": [""patient_id"", ""date""]}",{},Du ska plocka ut information som passar in i m...


In [15]:
print(raw_df.iloc[10]['text'])

Extern kontakt rörande patient 24-01-23 kl 1532, Läkare Erik Johansson:
Blir i egenskap av neurokirurgjour kontaktad av akutläkare på SkaS. Får information att det rör sig om en 63 årig man hemmahörande i Göteborgsregionen, känd hyperlipidemi och hypertension, rökare, i övrigt vital man som på julafton när han skulle ut och hämta tidningen på morgonen halkade på isen och slog i bakhuvudet. Fick ett stort blåmärke i bakhuvudet, hade initialt en del huvudvärk som dock gick över. Sökte ej vård. Inkommer nu till akutmottagningen SKaS med nedsatt kraft i vänster arm och ben samt tilltagande huvudvärk, enligt anhöriga har patienten även varit förvirrad och inte sig själv den senaste tiden. Patienten har varit på besök hos sina anhöriga i Skövde den senaste veckan och anhöriga har märkt att patienten blivit allt mer förvirrad och tog därför med honom till akuten. 
Enligt akutläkare är patienten vaken, RLS 1, dock förvirrad och vet inte var han är eller vilken dag det är. Något sluddrig i tale

In [ ]:
config = {'renormalize_logits': True, 'typical_p': 0.6168395262454404, 'temperature': 2.181784352404136, 'do_sample': True, 'epsilon_cutoff': 0.0006616580256429863}
response = requests.post('http://localhost:8000/set_cfg',headers={"Content-Type":"application/json"}, data=json.dumps(config))
print(response.json())

{'response': ':rocket:'}


In [4]:
with open("../Prompts/sums_reducer_sv.txt","r") as f:
    prompt_reducer = " ".join(f.readlines())

In [9]:
url = 'http://localhost:8000/llama_tokenize'
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
bertscore = evaluate.load("bertscore")
test_name = 'domain_adapt'

summary_df['patient_id'] = summary_df['patient_ids'].apply(lambda x: x[1:-1] if x[0]=='[' else x)
summary_df['date'] = summary_df['dates'].apply(lambda x:x[1:-1])
raw_df['patient_id'] = raw_df['patient_id'].astype(str) 
all_text = ""
#pandas_list = list()
answers = list()
references = list()
nAnswers = 0
start = time.time()
for index,r in summary_df.iterrows():
    temp_df = raw_df.loc[(raw_df['patient_id']==r['patient_id'])&(raw_df['date']==r['date'])]
    text = ""
    for row in temp_df.iterrows():
        row = row[1]
        if row['metadata'] != np.nan:
            text += str(row['metadata'])
        text +=row['text']
    if text == "":
        continue
    
    all_text += "\nGold standard:\n"
    all_text += r['text']
    data = {"text": text,"prompt_template":prompt_reducer,"prompt_reduce":prompt_reducer}

    headers = {"Content-Type":"application/json"}
    
    response = requests.post(url,headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        all_text += "\nSvar: \n"
        response = response.json()["response"]
        all_text += response
        answers.append(response)
        references.append(r['text'])
        if index % 10 == 0:
            print("\nGold standard:\n",r['text'])
            print('\nSvar:\n')      
            print(response)
        #result = scorer.score(response,joined['text_x'].iloc[0])
        #rouge_scores.append(result)
        #pandas_list.append([prompt,joined['text_x'].iloc[0],response,result['rouge1'].fmeasure,result['rouge2'].fmeasure,result['rougeL'].fmeasure])
        nAnswers += 1
    else:
        print('Response failed!')
        print("Status code: ", response.status_code)
        e = response.json()
        print("Error: ", e['error'])
        print("input_length ", e['input_len'])
        print("prompt:" ,prompt)
        print("tokenized prompt len ", e['prompt_len'])
        print("anamnesis: ", e['text'])
        print("here:", e['here'])
        print("reduce_index:",e['reduce_index'])
        break
        #'input_len':text_len,'prompt_len':prompt_template_tokenized_len,'prompt':prompt,'p_len':len(inputs[0]
        
        
#df_final = pd.DataFrame(data=pandas_list,columns=['Instruction','Gold Standard','Generated summary','Rouge 1','Rouge 2','Rouge L'])
#book = load_workbook(mock_data_path)
#writer = pd.ExcelWriter(mock_data_path, engine = 'openpyxl')
#writer.book = book
#df_final.to_excel(writer, sheet_name = test_name)
#writer.close()

end = time.time()
avg_iter = (end-start)/nAnswers
print("Average time per iteration:", avg_iter)
bleu_score = bleu.compute(predictions = answers, references = references)
rouge_score = rouge.compute(predictions = answers, references = references)
meteor_score = meteor.compute(predictions = answers, references = references)
bert_score = bertscore.compute(predictions = answers, references = references,lang='sv')

print(bleu_score)
print(rouge_score)
print(meteor_score)
print('bert score:',np.mean(bert_score['f1']))

with open(test_name + 'all_responses.txt','w') as f:
    f.write(all_text)
with open(test_name+'all_responses_metrics.txt','w') as f:
    f.write(f"{bleu_score} \n {rouge_score} \n {meteor_score}")

[nltk_data] Downloading package wordnet to /workspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /workspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /workspace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



Gold standard:
 *Sjukdomshistoria*:
Hypertoni, Hyperlipidemi, tidigare rökare (slutade för 10 år sedan). Fadern hade hjärt-kärlsjukdomar inklusive Abdominalt Aortaaneurysm

*Sökorsaker*:
Bukaortaneurysm på 68 mm

*Åtgärder*:
ABI

Svar:

  *Sjukdomshistoria (Patientens diagnoser, sjukdomshistorik och riskfaktorer (t.ex. sjukdomar i familjen))*

* Diagnoser: Misstänkt njurstensanfall, Tablettbehandlad Hypertoni, Hyperlipidemi
* Sjukdomshistorik: Tidigare ingenjör, Pensionär, Fadern hade hjärt-kärlsjukdomar inklusive Abdominalt Aortaaneurysm
* Riskfaktorer: Inga kända läkemedelsallergier

*Sökorsaker (Patientens symtom och/eller datum för ingrepp)*

* Symtom: Misstänkt njurstensanfall
* Datum för ingrepp: Aktuellt besök på AKOM med kontraströntgen som bekräftat ett bukaortaneurysm på 68 mm

*Åtgärder (Planerade undersökningar, behandlingar och åtgärder)*

* Planerade undersökningar: Utvärdering av bukaortaneurysm av kärlkirurgen
* Planerade behandlingar: Fortsätter hantering av hypertoni

In [7]:
url = 'http://localhost:8000/llama_tokenize'
prompt = """Besök neurodagvård, NPH team 23-11-02, Läk Mats Renberg:
Besökstyp: Nybesök NPH team
Mottagning: NPH mottagning neurodagvård
Tidigare/nuvarande sjukdomar: Förmaksflimmer Pradaxa behandlat, tidigare hjärtinfarkt, hypertension, Diabetes mellitus typ 2.
Aktuellt:
För bakgrund vänligen se besöksanteckning av Dr Dahlin 23-06-12. Inkommer med sin hustru för bedömning ny bedömning av neurostatus, bedömning av neuropsykolog, fysioterapeut samt lumbalpunktion/LP med tapptest/likvordynamiskt test.
Patient och hustru upplever en fortsatt försämring i både kisseriet, gångproblemen och framför allt minnet. Patienten har under hösten allt mer tappat sin egna initiativförmåga. Framstår apatisk enligt hustru av och till. Trött, sover mycket under dagen. Sover både middagslur och somnar till framför TVn. Sover längre under natten, tidigare ca 8 timmar, nu 10-12 h dagligen. Gällande kisseriet har patienten nu börjat använda blöja pga inkontinens. Fått en rullator via vårdcentralen för sin gångproblematik. Inga fler fall efter att han fått denna. Inga andra nytillkomna symptom.
Objektivt:
Allmäntillstånd: Vaken, framstår ej akut påverkad men kroniskt präglad.
Neurostatus: Väsentligen oförändrat jmf med tidigare undersökning av Dr Dahlin. Ej orienterad i tid eller rum, men i personnummer. Inget tecken till dysartri eller dysfasi. Kranialnerver 2-12 testade utan anmärkning. God kraft i armar och ben bilateralt, normal sensorik vid beröring. Nu måttlig paratoni i benen bilateralt. Finger-näsa-finger test och knä-häl test utan anmärkning. Normala intakta reflexer i biceps, triceps, brachioradialis, patella och achilles. Neg babinski. Tydlig bredspåriga gång små steg, vändning på 7-8 steg. Faller bakåt vid Rhombergs test.
Kognitiva test: MMSE 22 poäng.
Kommentar till undersökning: CT och MR hjärna som visar radiologisk bild av NPH.
Bedömning/åtgärd: Således 78 årig man, FF, HT, Dm typ 2, som inkommer för bedömning av misstänkt NPH. MMSE har sedan besöket i neurologimottagningen försämrats. Även tecken till försämring i neurostatus.
Plan:
Inväntar bedömning via neuropsykolog och fysioterapeut
Tapptest under eftermiddagen.

Kommentar till undersökning, 23-11-02, Läk Mats Renberg:
Lumbalpunktion:
Patienten identifieras med namn och personnummer. Informeras om procedur och evt risk. Det föreligger inga kontraindikationer till ingreppet. Patienten har pausat NOAK/Pradaxa 2 dygn innan ingreppet. Patienten placeras i sidoliggande ställning med ryggen helt ut. Lokaliserar L4-L5 område genom palpation av crista iliaca anterior superior. Rengör med klorhexidin jod. Bedövar med Xylocain 10 mg/ml ca 3-4 ml. Använder en 22 Gauge atraumatisk nål och för in den i lätt kraniell riktning och får klart likvorutbyte på första försöket. Tryck 17 cm. Tappar patienten på 40 ml likvor, skickar prov för demensmarkörer och infektion. Proceduren avslutas. Patienten informeras om evt post lumbalpunktions huvudvärk och får vänta sig till vården vid ihållande kraftig huvudvärk som är ihållande efter 3 dagar.
Genomför innan och efter lumbalpunktion Bingleys minnestest där patienten presterar ca 35% bättre efter tapptest."""
prompt_len = len(prompt)
data = {"text":prompt,"prompt_template":prompt_reducer,"prompt_reduce":prompt_reducer}

headers = {"Content-Type":"application/json"}
response = requests.post(url,headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print('Svar: ')
    #all_text += "Svar: \n"
    #print(prompt)
    #raise
    response = response.json()["response"]
    print(response)
else:
    print(response.json()['error'])

Svar: 
  *Sjukdomshistoria (Patientens diagnoser, sjukdomshistorik och riskfaktorer (t.ex. sjukdomar i familjen))*

* Diagnoser: Misstänkt NPH (neurocytologiskt växande hjärna)
* Sjukdomshistorik: Förmaksflimmer behandlat med Pradaxa, tidigare hjärtinfarkt, hypertension, Diabetes mellitus typ 2
* Riskfaktorer: Familjehistoria av NPH

*Sökorsaker (Patientens symtom och/eller datum för ingrepp)*

* Besök datum: 23-11-02
* Besökstyp: Nybesök NPH team
* Symtom: Försämring i både kisseriet, gångproblemer, minne och orientering
* Diagnostisering: CT och MR hjärna visar radiologisk bild av NPH
* Planerad behandling: Tapptest under eftermiddagen, neuropsykolog och fysioterapeut

*Åtgärder (Planerade undersökningar, behandlingar och åtgärder)*

* Planerad behandling:
	+ Neuropsykolog och fysioterapeut
	+ Tapptest under eftermiddagen
	+ CT och MR hjärna för diagnostisering av NPH
	+ Lumbalpunktion med risk för huvudvärk och eventuell post lumbalpunktions huvudvärk

Note: The information has been

In [64]:
df_final.head(10)

NameError: name 'df_final' is not defined

In [65]:
print(np.mean([x['rouge1'].fmeasure for x in rouge_scores]))
print(np.mean([x['rouge2'].fmeasure for x in rouge_scores]))
print(np.mean([x['rougeL'].fmeasure for x in rouge_scores]))

NameError: name 'rouge_scores' is not defined

In [50]:
print([x['rougeL'].fmeasure for x in rouge_scores])

[0.19685039370078738, 0.16445623342175064, 0.2710997442455243, 0.1769911504424779]


In [47]:
from datasets import load_from_disk
train_test_valid_dataset = load_from_disk('/workspace/data/Gabriel/pubmed_swe')
val_set = train_test_valid_dataset['valid']
val_set = val_set.train_test_split(test_size=0.1111)['test']

In [50]:
# Create a timezone with +1 hour offset from UTC
timezone_offset = timezone(timedelta(hours=1))
url = 'http://localhost:8000/llama_tokenize'
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
test_name = 'LLama-2'

answers = []
references = []

with open("../Prompts/chunk_to_sum_en.txt","r") as f:
    prompt = " ".join(f.readlines())

with open("../Prompts/sums_reducer_en.txt","r") as f:
    prompt_reducer = " ".join(f.readlines())
    
for i,item in enumerate(val_set):
    prompt_len = len(prompt)
    data = {"prompt_template":prompt,"prompt_reduce":prompt_reducer ,"text":item['article']}

    headers = {"Content-Type":"application/json"}
    response = requests.post(url,headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        references.append(item["abstract"])
        #print('Svar: ')
        #all_text += "Svar: \n"
        #print(prompt)
        #raise
        response = response.json()["response"]
        answers.append(response)
        #result = scorer.score(response,joined['text_x'].iloc[0])
        #rouge_scores.append(result)
        #pandas_list.append([prompt,joined['text_x'].iloc[0],response,result['rouge1'].fmeasure,result['rouge2'].fmeasure,result['rougeL'].fmeasure])
    
        if i % 5 == 0:
            # Get current time adjusted for timezone (+1 from UTC)
            adjusted_time_now = datetime.now(timezone_offset)

            # Print the adjusted time
            print(i,adjusted_time_now.strftime('%Y-%m-%d %H:%M:%S %Z%z'))
            if i % 10 == 0:
                print("=================")
                print(i)
                print(item["abstract"])
                print("-----------------")
                print(response)
                print("=================")
                if i ==100:
                    break
    else:
        print('Response failed!')
        print("Status code: ", response.status_code)
        print("Error: ", response.json()['error'])

        
bleu_score = bleu.compute(predictions = answers, references = references)
rouge_score = rouge.compute(predictions = answers, references = references)
meteor_score = meteor.compute(predictions = answers, references = references)

print(bleu_score)
print(rouge_score)
print(meteor_score)

[nltk_data] Downloading package wordnet to /workspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /workspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /workspace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0 2024-02-16 13:48:44 UTC+01:00+0100
0
operationsfarkosten har alltid förlitat sig på användning av instrument. 
 Innovationer inom kirurgi har parallellt med innovationer inom instrumentering. 
 framsteg i kirurgisk instrumentering fortsätter idag och har möjliggjort enorma framsteg i kirurgiska ingrepp och resultat under denna generation. 
 datorer och relaterad teknik nu ändrar gränssnittet mellan kirurgen och patienten, och är redo att förbättra patientens resultat genom att förbättra kirurgens färdigheter och utbildning. 
 tillämpning av datorförstärkta telemanipulatorer, eller robotar 
 , kan specifikt förbättra verksamheten, till exempel heller myotomi, som kräver god visualisering och exakt noggrann dissektion av känsliga strukturer. 
 denna översyn omfattar patofysiologi av achalasia och dess historia av medicinsk och kirurgisk behandling, vilket leder till moderna robotik telekirurgiska metoder. 
 Förbättringar i resultat från medicinsk till standard kirurgisk till robotisk t

KeyboardInterrupt: 